In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/config.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/tokenizer.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/metadata.json
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/model.weights.h5
/kaggle/input/gemma/keras/gemma_instruct_2b_en/2/assets/tokenizer/vocabulary.spm
/kaggle/input/llm-prompt-recovery-synthetic-datastore/gemma1000_w7b.csv
/kaggle/input/3000-rewritten-texts-prompt-recovery-challenge/prompts_0_500_wiki_first_para_3000.csv
/kaggle/input/3000-rewritten-texts-prompt-recovery-challenge/rewrite_prompts.csv
/kaggle/input/llm-prompt-recovery/sample_submission.csv
/kaggle/input/llm-prompt-recovery/train.csv
/kaggle/input/llm-prompt-recovery/test.csv


In [2]:
import pandas as pd

# GENERATE TRAIN DATA

In [26]:
messages=pd.read_csv('/kaggle/input/3000-rewritten-texts-prompt-recovery-challenge/prompts_0_500_wiki_first_para_3000.csv')

In [27]:
messages.tail()

,original_text,rewrite_prompt,rewritten_text
2995,Výškovce is a village and municipality in Stro...,Write it as the last chapter of a book that ch...,## The Whispering Walls of Výškovce\n\nThe win...
2996,The World I Want to Leave Behind is the fourth...,Adapt it as a solemn monastic chant.,"Sure, here's the adapted chant:\n\n""The world ..."
2997,The Akademie Rudolph-Antoniana was an early mo...,Convert it into a narrative of the first sunri...,A pall of darkness cast the Akademie Rudolph-A...
2998,"A prisoner transport vehicle, informally known...",Style it as a proclamation by a newly crowned ...,"**Hear ye, hear ye, gathered mortals,**\n\nI, ..."
2999,Triadic-line poetry or stepped line is a long ...,Convert the text into a series of pictographs ...,"Sure, here's the text converted into pictograp..."


In [29]:
rewrite_prompts = [
    'Explain this to me like I\'m five.',
    'Convert this into a sea shanty.',
    'Make this rhyme.',
]

In [ ]:
import random
random.seed(0)
# This is the prompt format the model expects
USER_CHAT_TEMPLATE = "<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n"

rewrite_data = []

for original in original_text:
    rewrite_prompt = random.choice(rewrite_prompts)
    prompt = f'{rewrite_prompt}\n{original}'
    rewritten_text = model.generate(
        USER_CHAT_TEMPLATE.format(prompt=prompt),
        device=device,
        output_len=100,
    )
    rewrite_data.append({
        'original_text': original,
        'rewrite_prompt': rewrite_prompt,
        'rewritten_text': rewritten_text,
    })

In [ ]:
rewrite_data_df = pd.DataFrame(rewrite_data)
rewrite_data_df[:1].values

# MODEL BUILDING

In [4]:
import os

In [5]:
os.environ["KERAS_BACKEND"]="jax"
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00"
import keras
import keras_nlp

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

import plotly.graph_objs as go
import plotly.express as px
from IPython.display import display, Markdown

2024-03-15 07:14:37.847165: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 07:14:37.847234: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-15 07:14:37.848727: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
class CFG:
    seed = 42
    dataset_path = "/kaggle/input/llm-prompt-recovery"
    preset = "gemma_instruct_2b_en" # name of pretrained Gemma
    sequence_length = 512 # max size of input sequence for training
    batch_size = 1 # size of the input batch in training
    epochs = 1 # number of epochs to train

In [7]:
keras.utils.set_random_seed(CFG.seed)

In [8]:
df1=pd.read_csv('/kaggle/input/llm-prompt-recovery-synthetic-datastore/gemma1000_w7b.csv')
df1=df1[['original_text','rewrite_prompt','gemma_7b_rewritten_text_temp0']]
df1=df1.rename(columns={'gemma_7b_rewritten_text_temp0':'rewritten_text'})

df2=pd.read_csv('/kaggle/input/3000-rewritten-texts-prompt-recovery-challenge/prompts_0_500_wiki_first_para_3000.csv')

df=pd.concat([df1,df2],axis=0)
df=df.sample(2000).reset_index(drop=True)
df.head(5)

,original_text,rewrite_prompt,rewritten_text
0,Story highlights New findings suggest sperm al...,Turn this into a programmer's code.,```python\n# Code to summarize the text\n\nspe...
1,Glasgow's Needy is a non-registered charity or...,Make this a formal apology letter to a customer.,"[Your Name]\n[Your Address]\nGlasgow, Scotland..."
2,"Pune: In a bizarre case, a parrot accused of ""...",Convert this into a technique to be used.,**Technique:**\n\n**Step 1: Gather evidence.**...
3,Berrya is a genus of evergreen trees with fibr...,Convert the text into the layout of a classic ...,## Level Layout: Berrya Forest\n\n**Background...
4,The Libertarian Party of Oregon is a political...,Recast it as the backstory for a mysterious an...,In the shadows of the Evergreen State of Orego...


In [9]:
template = """Instruction:\nBelow, the `Original Text` passage has been rewritten/transformed/improved into `Rewritten Text` by the `Gemma 7b-it` LLM with a certain prompt/instruction. Your task is to carefully analyze the differences between the `Original Text` and `Rewritten Text`, and try to infer the specific prompt or instruction that was likely given to the LLM to rewrite/transform/improve the text in this way.\n\nOriginal Text:\n{original_text}\n\nRewriten Text:\n{rewritten_text}\n\nResponse:\n{rewrite_prompt}"""

In [10]:
df["prompt"] = df.progress_apply(lambda row: template.format(original_text=row.original_text,
                                                             rewritten_text=row.rewritten_text,
                                                             rewrite_prompt=row.rewrite_prompt), axis=1)
data = df.prompt.tolist()

  0%|          | 0/2000 [00:00<?, ?it/s]

In [11]:
def colorize_text(text):
    for word, color in zip(["Instruction", "Original Text", "Rewriten Text", "Response"],
                           ["red", "yellow", "blue", "green"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [12]:
sample = data[10]

sample = colorize_text(sample)

display(Markdown(sample))




**<font color='red'>Instruction:</font>**
Below, the `Original Text` passage has been rewritten/transformed/improved into `Rewritten Text` by the `Gemma 7b-it` LLM with a certain prompt/instruction. Your task is to carefully analyze the differences between the `Original Text` and `Rewritten Text`, and try to infer the specific prompt or instruction that was likely given to the LLM to rewrite/transform/improve the text in this way.



**<font color='yellow'>Original Text:</font>**
Zoltán Tóth (born 24 August 1979) is a Hungarian former competitive figure skater. He is a five-time Hungarian national champion and competed in two Winter Olympics.



**<font color='blue'>Rewriten Text:</font>**
Sure, here's the rewritten text as if it were a list of ingredients in a magical potion:

The ingredients to create the potion:

Zoltán Tóth's birth date: 24 August 1979
Hungarian heritage
Five-time Hungarian national champion
Two Winter Olympics participation



**<font color='green'>Response:</font>**
Rewrite the text as if it were a list of ingredients in a magical potion.

In [13]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset(CFG.preset)
gemma_lm.summary()

Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_instruct_2b_en/2' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

In [14]:
x, y, sample_weight = gemma_lm.preprocessor(data[0:2])

In [15]:
for k, v in x.items():
    print(k, ":", v.shape)

token_ids : (2, 8192)
padding_mask : (2, 8192)


# INFERENCE FROM MODEL

In [16]:
row = df.iloc[10]

prompt = template.format(
    original_text=row.original_text,
    rewritten_text=row.rewritten_text,
    rewrite_prompt="",
)

output = gemma_lm.generate(prompt, max_length=512)

output = colorize_text(output)

display(Markdown(output))



**<font color='red'>Instruction:</font>**
Below, the `Original Text` passage has been rewritten/transformed/improved into `Rewritten Text` by the `Gemma 7b-it` LLM with a certain prompt/instruction. Your task is to carefully analyze the differences between the `Original Text` and `Rewritten Text`, and try to infer the specific prompt or instruction that was likely given to the LLM to rewrite/transform/improve the text in this way.



**<font color='yellow'>Original Text:</font>**
Zoltán Tóth (born 24 August 1979) is a Hungarian former competitive figure skater. He is a five-time Hungarian national champion and competed in two Winter Olympics.



**<font color='blue'>Rewriten Text:</font>**
Sure, here's the rewritten text as if it were a list of ingredients in a magical potion:

The ingredients to create the potion:

Zoltán Tóth's birth date: 24 August 1979
Hungarian heritage
Five-time Hungarian national champion
Two Winter Olympics participation



**<font color='green'>Response:</font>**
The LLM was likely given the prompt or instruction to create a list of ingredients for a magical potion, based on the information in the original text.

In [17]:
row = df.iloc[20]

prompt = template.format(
    original_text=row.original_text,
    rewritten_text=row.rewritten_text,
    rewrite_prompt="",
)

output = gemma_lm.generate(prompt, max_length=512)

output = colorize_text(output)

display(Markdown(output))



**<font color='red'>Instruction:</font>**
Below, the `Original Text` passage has been rewritten/transformed/improved into `Rewritten Text` by the `Gemma 7b-it` LLM with a certain prompt/instruction. Your task is to carefully analyze the differences between the `Original Text` and `Rewritten Text`, and try to infer the specific prompt or instruction that was likely given to the LLM to rewrite/transform/improve the text in this way.



**<font color='yellow'>Original Text:</font>**
Abu al-Hassan Ali ibn Moussa ibn Rashid al-Alami (), also known as Sherif Moulay Ali Ben Rachid, was the founder of the city of Chefchaouen, Morocco. He was an Idrisid and descendant of Sufi saint Abd as-Salam ibn Mashish al-Alami. He was also the father of Sayyida al-Hurra, governor of Tetouan.



**<font color='blue'>Rewriten Text:</font>**
## Internal Audit Checklist - Governance and Historical Background

**Objective:** To assess the accuracy and completeness of the historical narrative regarding the founding of Chefchaouen and the related individuals' identities and achievements.

**Criteria:**

**1. Accuracy of the historical narrative:**

- Does the text accurately describe Abu al-Hassan Ali ibn Moussa ibn Rashid al-Alami's role as the founder of Chefchaouen?
- Does the text accurately describe the Idrisid heritage and Sufi lineage of the founder?
- Is the information about the founder's relationship to Sayyida al-Hur



**<font color='green'>Response:</font>**
**Yes, the rewritten text is largely accurate and provides a good overview of the historical narrative.**

**2. Completeness of the historical narrative:**

- Does the text provide all the relevant historical information about the founding of Chefchaouen?
- Are there any missing or omitted key details?
- Is the text well-organized and easy to follow?



**<font color='green'>Response:</font>**
**The rewritten text provides a good overview of the historical narrative, but there are a few minor omissions and missing details that could be further addressed for greater accuracy.**

**3. Clarity and conciseness of the rewritten text:**

- Is the text written in a clear and concise style?
- Is the information presented in a logical and easy-to-understand manner?
- Does the text use appropriate vocabulary and tone?



**<font color='green'>Response:</font>**
**The rewritten text is well-written and clear, providing a concise and accurate overview of the historical narrative.**

**4. Alignment with the original text

# FINE TUNING WITH LoRA

In [18]:
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,507,536,384 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,507,536,384 (9.34 GB)

 Trainable params: 1,363,968 (5.20 MB)

 Non-trainable params: 2,506,172,416 (9.34 GB)

In [19]:
gemma_lm.preprocessor.sequence_length = CFG.sequence_length 

# Compile the model with loss, optimizer, and metric
gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=3e-5),
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train model
gemma_lm.fit(data, epochs=CFG.epochs, batch_size=CFG.batch_size)

2000/2000 ━━━━━━━━━━━━━━━━━━━━ 1486s 732ms/step - loss: 1.3144 - sparse_categorical_accuracy: 0.6193


# INFERENCE AFTER FINE TUNING

In [20]:
row = df.iloc[10]

prompt = template.format(
    original_text=row.original_text,
    rewritten_text=row.rewritten_text,
    rewrite_prompt="",
)


output = gemma_lm.generate(prompt, max_length=512)


output = colorize_text(output)

display(Markdown(output))



**<font color='red'>Instruction:</font>**
Below, the `Original Text` passage has been rewritten/transformed/improved into `Rewritten Text` by the `Gemma 7b-it` LLM with a certain prompt/instruction. Your task is to carefully analyze the differences between the `Original Text` and `Rewritten Text`, and try to infer the specific prompt or instruction that was likely given to the LLM to rewrite/transform/improve the text in this way.



**<font color='yellow'>Original Text:</font>**
Zoltán Tóth (born 24 August 1979) is a Hungarian former competitive figure skater. He is a five-time Hungarian national champion and competed in two Winter Olympics.



**<font color='blue'>Rewriten Text:</font>**
Sure, here's the rewritten text as if it were a list of ingredients in a magical potion:

The ingredients to create the potion:

Zoltán Tóth's birth date: 24 August 1979
Hungarian heritage
Five-time Hungarian national champion
Two Winter Olympics participation



**<font color='green'>Response:</font>**
Transform this into a list of ingredients for a magical potion.

In [21]:
row = df.iloc[20]

prompt = template.format(
    original_text=row.original_text,
    rewritten_text=row.rewritten_text,
    rewrite_prompt="",
)

output = gemma_lm.generate(prompt, max_length=512)

output = colorize_text(output)

display(Markdown(output))



**<font color='red'>Instruction:</font>**
Below, the `Original Text` passage has been rewritten/transformed/improved into `Rewritten Text` by the `Gemma 7b-it` LLM with a certain prompt/instruction. Your task is to carefully analyze the differences between the `Original Text` and `Rewritten Text`, and try to infer the specific prompt or instruction that was likely given to the LLM to rewrite/transform/improve the text in this way.



**<font color='yellow'>Original Text:</font>**
Abu al-Hassan Ali ibn Moussa ibn Rashid al-Alami (), also known as Sherif Moulay Ali Ben Rachid, was the founder of the city of Chefchaouen, Morocco. He was an Idrisid and descendant of Sufi saint Abd as-Salam ibn Mashish al-Alami. He was also the father of Sayyida al-Hurra, governor of Tetouan.



**<font color='blue'>Rewriten Text:</font>**
## Internal Audit Checklist - Governance and Historical Background

**Objective:** To assess the accuracy and completeness of the historical narrative regarding the founding of Chefchaouen and the related individuals' identities and achievements.

**Criteria:**

**1. Accuracy of the historical narrative:**

- Does the text accurately describe Abu al-Hassan Ali ibn Moussa ibn Rashid al-Alami's role as the founder of Chefchaouen?
- Does the text accurately describe the Idrisid heritage and Sufi lineage of the founder?
- Is the information about the founder's relationship to Sayyida al-Hur



**<font color='green'>Response:</font>**
Turn this into a proposal for a historical audit.

# TEST AND SUBMISSION

In [22]:
test_df = pd.read_csv("/kaggle/input/llm-prompt-recovery/test.csv")
test_df['original_text'] = test_df['original_text'].fillna("")
test_df['rewritten_text'] = test_df['rewritten_text'].fillna("")
test_df.head()

,id,original_text,rewritten_text
0,-1,The competition dataset comprises text passage...,Here is your shanty: (Verse 1) The text is rew...


In [23]:
row = test_df.iloc[0]

# Generate Prompt using template
prompt = template.format(
    original_text=row.original_text,
    rewritten_text=row.rewritten_text,
    rewrite_prompt="",
)

# Infer
output = gemma_lm.generate(prompt, max_length=512)

# Colorize
output = colorize_text(output)

# Display in markdown
display(Markdown(output))



**<font color='red'>Instruction:</font>**
Below, the `Original Text` passage has been rewritten/transformed/improved into `Rewritten Text` by the `Gemma 7b-it` LLM with a certain prompt/instruction. Your task is to carefully analyze the differences between the `Original Text` and `Rewritten Text`, and try to infer the specific prompt or instruction that was likely given to the LLM to rewrite/transform/improve the text in this way.



**<font color='yellow'>Original Text:</font>**
The competition dataset comprises text passages that have been rewritten by the Gemma LLM according to some rewrite_prompt instruction. The goal of the competition is to determine what prompt was used to rewrite each original text.  Please note that this is a Code Competition. When your submission is scored, this example test data will be replaced with the full test set. Expect roughly 2,000 original texts in the test set.



**<font color='blue'>Rewriten Text:</font>**
Here is your shanty: (Verse 1) The text is rewritten, the LLM has spun, With prompts so clever, they've been outrun. The goal is to find, the prompt so bright, To crack the code, and shine the light. (Chorus) Oh, this is a code competition, my dear, With text and prompts, we'll compete. Two thousand texts, a challenge grand, To guess the prompts, hand over hand.(Verse 2) The original text, a treasure lost, The rewrite prompt, a secret to be



**<font color='green'>Response:</font>**
Write this as a shanty and submit it as the response.

In [24]:
preds = []
for i in tqdm(range(len(test_df))):
    row = test_df.iloc[i]

    # Generate Prompt using template
    prompt = template.format(
        original_text=row.original_text,
        rewritten_text=row.rewritten_text,
        rewrite_prompt=""
    )

    # Infer
    output = gemma_lm.generate(prompt, max_length=512)
    pred = output.replace(prompt, "") # remove the prompt from output
    
    # Store predictions
    preds.append([row.id, pred])

  0%|          | 0/1 [00:00<?, ?it/s]

In [25]:
sub_df = pd.DataFrame(preds, columns=["id", "rewrite_prompt"])
sub_df['rewrite_prompt'] = sub_df['rewrite_prompt'].fillna("")
sub_df['rewrite_prompt'] = sub_df['rewrite_prompt'].map(lambda x: "Improve the essay" if len(x) == 0 else x)
sub_df.to_csv("submission.csv",index=False)
sub_df.head()

,id,rewrite_prompt
0,-1,Turn this into a shanty.
